In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# 1.Thu thập dữ liệu

## Dữ liệu sử dụng
Thống kê dữ liệu theo từng ngày các trường hợp mắc bệnh, tử vong, hồi phục từ 22/1/2020 - 20/6/2021
<br>
Các dữ liệu đều được lấy từ kho dữ liệu COVID-19 của Trung tâm Khoa học và Kỹ thuật Hệ thống (CSSE) thuộc Đại học Johns Hopkins (USA)
<br>
Đường link lấy dữ liệu: https://github.com/CSSEGISandData/COVID-19
<br>
Điều kiện để được sử dụng dữ liệu: Trích dẫn 1 đoạn mà người public dữ liệu yêu cầu (Đã trích dẫn ở cuối bài làm)
<br>
Người ta thu thập dữ liệu bằng cách lấy thông tin từ 1 số lượng lớn các trang web. Đa số thì sẽ lấy thông tin từ từng Quốc Gia

In [2]:
confirm_df = pd.read_csv('time_series_covid19_confirmed_global.csv')
death_df = pd.read_csv('time_series_covid19_deaths_global.csv')
recover_df = pd.read_csv('time_series_covid19_recovered_global.csv')
confirm_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/11/21,6/12/21,6/13/21,6/14/21,6/15/21,6/16/21,6/17/21,6/18/21,6/19/21,6/20/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,87716,88740,89861,91458,93272,93288,96531,98734,98734,98734
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,132437,132449,132459,132461,132469,132476,132481,132484,132488,132490
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,132727,133070,133388,133742,134115,134458,134840,135219,135586,135821
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,13813,13813,13813,13826,13828,13836,13839,13842,13842,13842
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,36455,36600,36705,36790,36921,37094,37289,37467,37604,37678


## Tiền xử lý
Gom các cột ngày tháng vào 1 cột có tên là Date, tạo ra 1 bảng duy nhất thể hiện tổng số ca mắc, hồi phục và tử vong của các nước theo ngày

In [3]:
dates = confirm_df.iloc[:,4:]
#Không lấy các cột lat và long vì chúng ta không có dự định biểu thị vị trí trên bản đồ thế giới
confirm_df_melt = confirm_df.melt(id_vars=['Province/State', 'Country/Region'], value_vars=dates, var_name='Date', value_name='Confirmed')
death_df_melt = death_df.melt(id_vars=['Province/State', 'Country/Region'], value_vars=dates, var_name='Date', value_name='Deaths')
recover_df_melt = recover_df.melt(id_vars=['Province/State', 'Country/Region'], value_vars=dates, var_name='Date', value_name='Recovered')

In [4]:
#Ghép các giá trị số ca đã xác nhận, số ca tử vong và số ca hồi phục vào dataframe
covid19_df = pd.merge(left=confirm_df_melt, right=death_df_melt, how='left', on=['Province/State', 'Country/Region', 'Date'])
covid19_df = pd.merge(left=covid19_df, right=recover_df_melt, how='left', on=['Province/State', 'Country/Region', 'Date'])
covid19_df.head()

,Province/State,Country/Region,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,1/22/20,0,0,0.0
1,NaN,Albania,1/22/20,0,0,0.0
2,NaN,Algeria,1/22/20,0,0,0.0
3,NaN,Andorra,1/22/20,0,0,0.0
4,NaN,Angola,1/22/20,0,0,0.0


# 2.Khám phá dữ liệu

In [6]:
covid19_df.shape

(143448, 6)

In [7]:
have_duplicated_rows = covid19_df.duplicated().any()
have_duplicated_rows

False

## Số dòng, cột, ý nghĩa của từng dòng và kiểm tra việc lặp
<br>
Dữ liêu có 142336 dòng và 6 cột. Mỗi dòng biểu thị số ca mắc, số ca tử vong và số ca hồi phục tổng đã được xác nhận tại thời điểm thu thập dữ liệu tại Quốc gia (+ Địa điểm) đó
<br>
Dữ liệu mỗi dòng đều có ý nghĩa giống nhau và không dòng nào bị lặp

## Chú thích dữ liệu (Ý nghĩa của từng cột)
__Province_State__: Tên tiểu bang, địa danh,...
<br>
__Country_Region__: Tên quốc gia
<br>
__Date__: Thời gian thu thập dữ liệu
<br>
__Confirmed__: Số ca mắc bệnh đã xác nhận
<br>
__Deaths__: Số ca tử vong đã được xác nhận
<br>
__Recovered__: Số ca hồi phục đã được xác nhận

In [8]:
covid19_df.dtypes

Province/State     object
Country/Region     object
Date               object
Confirmed           int64
Deaths              int64
Recovered         float64
dtype: object

## Kiểu dữ liệu
Cột __Province_State__, __Country_Region__, __Date__ có kiểu dữ liệu object cần xử lý tiếp tục
<br>
Cột __Confirmed__, __Deaths__, __Recovered__ có kiểu dữ liệu numeric. Để đồng hộ hoá thì chúng ta sẽ chuyển kiểu dữ liệu của __Recovered__ từ float64 về int64 như 2 cái trước.

In [10]:
def open_object_dtype(s):
    dtypes = set(s.apply(lambda x: type(x)).unique())
    return dtypes
print(open_object_dtype(covid19_df['Province/State']))
print(open_object_dtype(covid19_df['Country/Region']))
print(open_object_dtype(covid19_df['Date']))

{<class 'str'>, <class 'float'>}
{<class 'str'>}
{<class 'str'>}


In [11]:
covid19_df['Province/State'].unique()

array([nan, 'Australian Capital Territory', 'New South Wales',
       'Northern Territory', 'Queensland', 'South Australia', 'Tasmania',
       'Victoria', 'Western Australia', 'Alberta', 'British Columbia',
       'Diamond Princess', 'Grand Princess', 'Manitoba', 'New Brunswick',
       'Newfoundland and Labrador', 'Northwest Territories',
       'Nova Scotia', 'Nunavut', 'Ontario', 'Prince Edward Island',
       'Quebec', 'Repatriated Travellers', 'Saskatchewan', 'Yukon',
       'Anhui', 'Beijing', 'Chongqing', 'Fujian', 'Gansu', 'Guangdong',
       'Guangxi', 'Guizhou', 'Hainan', 'Hebei', 'Heilongjiang', 'Henan',
       'Hong Kong', 'Hubei', 'Hunan', 'Inner Mongolia', 'Jiangsu',
       'Jiangxi', 'Jilin', 'Liaoning', 'Macau', 'Ningxia', 'Qinghai',
       'Shaanxi', 'Shandong', 'Shanghai', 'Shanxi', 'Sichuan', 'Tianjin',
       'Tibet', 'Unknown', 'Xinjiang', 'Yunnan', 'Zhejiang',
       'Faroe Islands', 'Greenland', 'French Guiana', 'French Polynesia',
       'Guadeloupe', 'Martiniq

In [12]:
covid19_df['Date'] = pd.to_datetime(covid19_df['Date'], format = '%m/%d/%y')
covid19_df['Recovered'] = covid19_df['Recovered'].fillna(0)
covid19_df['Recovered'] = covid19_df['Recovered'].astype('int64')
covid19_df['Province/State'] = covid19_df[['Province/State','Country/Region']].replace(np.nan,'')

In [13]:
num_cols = ['Date','Confirmed','Deaths','Recovered']
def missing_ratio(df):
    return df.isna().mean() * 100
nume_col_profiles_df = covid19_df[num_cols].agg([missing_ratio, 'min', 'max'])
nume_col_profiles_df

,Date,Confirmed,Deaths,Recovered
missing_ratio,0.0,0.0,0.0,0.0
min,2020-01-22 00:00:00,0.0,0.0,0.0
max,2021-06-20 00:00:00,33541887.0,601824.0,28844199.0


## Dữ liệu numeric
<br>
Dữ liệu không có giá trị thiếu
<br>
Dữ liệu không có giá trị bất thường về ngày đầu tiên và ngày cuối cùng thu thập dữ liệu, số ca mắc, tử vong, hồi phục cao nhất tính tới ngày 20/6/2021

In [15]:
cat_cols = list(set(covid19_df.columns) - set(num_cols))
def missing_ratio(df):
    return df.isna().mean() * 100
def num_diff_vals(df):
    return df.nunique()
def diff_vals(df):
    return df.unique()
cate_col_profiles_df = covid19_df[cat_cols].agg([missing_ratio, num_diff_vals, diff_vals])
cate_col_profiles_df

,Country/Region,Province/State
missing_ratio,0.0,0.0
num_diff_vals,194,88
diff_vals,"[Afghanistan, Albania, Algeria, Andorra, Angol...","[, Australian Capital Territory, New South Wal..."


## Dữ liệu categorical
Dữ liệu không có giá trị thiếu
<br>
Dữ liệu có ít quốc gia trên thế giới hơn so với thực tế (194 quốc gia so với 204 quốc gia đã được xác nhận) ngoài ra không còn gì bất thường

# 3. Trả lời câu hỏi

## Câu hỏi
1. Những nước nào có tỉ lệ số người tử vong trên số người hồi phục thấp để chúng ta có thể học tập các biện pháp y tế của họ đồng thời những nước nào có tỉ lệ cao để chúng ta có thể rút kinh nghiệm từ những khó khăn của họ.
<br><br>
2. Trong vòng 1 tuần, 1 tháng qua, nước nào có số người nhiễm mới thấp so với thế giới. Từ đó chúng ta có thể học hỏi về cách phòng bệnh và mở cửa giao thương. Đồng thời chúng ta sẽ tìm những nước có người nhiễm cao để rút kinh nghiệm từ những khó khăn của họ và cảnh báo cho người dân không chủ quan.
<br><br>
3. Kiểm tra và thống kê dữ liệu của Việt Nam để chúng ta hiểu rõ hơn tình hình của chúng ta hiện tại để biết khả năng phòng chống và sự phức tạp của dịch bệnh này ở nước ta như thế nào.

# 4.Nhìn lại quá trình làm đồ án

# 5.Tài liệu tham khảo

Tài liệu pandas: https://pandas.pydata.org/docs/reference/index.html
<br>
Tài liệu về covid-19 của trường Đại học John Hopkins: https://github.com/CSSEGISandData/COVID-19

Dong E, Du H, Gardner L. An interactive web-based dashboard to track COVID-19 in real time. Lancet Inf Dis. 20(5):533-534. doi: 10.1016/S1473-3099(20)30120-1